In [1]:
import beepy

In [1]:
import tensorflow as tf 
import os 
import PIL
import pandas as pd 
import numpy as np 
import pathlib
import matplotlib.pyplot as plt 
import cv2
import warnings
import config
import re

In [2]:
data_dir = pathlib.Path(config.CHEXPERT_DATA_PATH)
os.listdir(data_dir)

['chexpert_train_image_paths.txt',
 'chexpert_valid_image_paths.txt',
 'train',
 'train.csv',
 'valid',
 'valid.csv']

In [3]:
train_df = pd.read_csv(os.path.join(data_dir, 'train.csv'))
valid_df = pd.read_csv(os.path.join(data_dir, 'valid.csv'))

In [4]:
train_df.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223414 entries, 0 to 223413
Data columns (total 19 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Path                        223414 non-null  object 
 1   Sex                         223414 non-null  object 
 2   Age                         223414 non-null  int64  
 3   Frontal/Lateral             223414 non-null  object 
 4   AP/PA                       191027 non-null  object 
 5   No Finding                  22381 non-null   float64
 6   Enlarged Cardiomediastinum  44839 non-null   float64
 7   Cardiomegaly                46203 non-null   float64
 8   Lung Opacity                117778 non-null  float64
 9   Lung Lesion                 11944 non-null   float64
 10  Edema                       85956 non-null   float64
 11  Consolidation               70622 non-null   float64
 12  Pneumonia                   27608 non-null   float64
 13  Atelectasis   

In [6]:
valid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Path                        234 non-null    object 
 1   Sex                         234 non-null    object 
 2   Age                         234 non-null    int64  
 3   Frontal/Lateral             234 non-null    object 
 4   AP/PA                       202 non-null    object 
 5   No Finding                  234 non-null    float64
 6   Enlarged Cardiomediastinum  234 non-null    float64
 7   Cardiomegaly                234 non-null    float64
 8   Lung Opacity                234 non-null    float64
 9   Lung Lesion                 234 non-null    float64
 10  Edema                       234 non-null    float64
 11  Consolidation               234 non-null    float64
 12  Pneumonia                   234 non-null    float64
 13  Atelectasis                 234 non

In [7]:
train_df.iloc[:5,0]

0    CheXpert-v1.0-small/train/patient00001/study1/...
1    CheXpert-v1.0-small/train/patient00002/study2/...
2    CheXpert-v1.0-small/train/patient00002/study1/...
3    CheXpert-v1.0-small/train/patient00002/study1/...
4    CheXpert-v1.0-small/train/patient00003/study1/...
Name: Path, dtype: object

#### Replacing NaN values for the 14 labels

In [8]:
# Replacing all the NaN values in train and validation dataframes with 0.0 for the 14 labels
label_columns = list(train_df.iloc[:5, 5:].columns)
label_columns

['No Finding',
 'Enlarged Cardiomediastinum',
 'Cardiomegaly',
 'Lung Opacity',
 'Lung Lesion',
 'Edema',
 'Consolidation',
 'Pneumonia',
 'Atelectasis',
 'Pneumothorax',
 'Pleural Effusion',
 'Pleural Other',
 'Fracture',
 'Support Devices']

In [9]:
train_df.loc[:, label_columns] = train_df.loc[:, label_columns].replace(to_replace = np.nan, value = 0.0)

valid_df.loc[:, label_columns] = valid_df.loc[:, label_columns].replace(to_replace = np.nan, value= 0.0)

In [10]:
# Replacing uncertain labels with positive labels
def uncertain_to_pos(df):
    df.replace(to_replace = -1.0, value = 1.0, inplace = True)

uncertain_to_pos(train_df)
uncertain_to_pos(valid_df)

In [11]:
train_df.set_index('Path', inplace = True)
valid_df.set_index('Path', inplace = True)


### Creating a data pipeline

In [12]:
with open('chexpert_train_image_paths.txt', 'r') as file:
    train_image_paths = file.readline()
with open('chexpert_valid_image_paths.txt', 'r') as file:
    valid_image_paths = file.readline()

In [13]:
import ast
train_image_paths = ast.literal_eval(train_image_paths)
valid_image_paths = ast.literal_eval(valid_image_paths)

In [14]:
def modify_path(path):
    parts = path.split('/')[4:]
    new_path = os.path.join(data_dir, os.path.join(*parts))
    return new_path

In [15]:
from joblib import Parallel, delayed

In [16]:
train_image_paths = Parallel(n_jobs= 10, backend = 'threading')(delayed(modify_path)(path) for path in train_image_paths)

In [17]:
valid_image_paths = Parallel(n_jobs= 10, backend = 'threading')(delayed(modify_path)(path) for path in valid_image_paths)

In [18]:
train_image_paths[:5]

['D:\\Datasets\\CheXpert-v1.0-small\\train\\patient00734\\study3\\view2_lateral.jpg',
 'D:\\Datasets\\CheXpert-v1.0-small\\train\\patient00734\\study3\\view1_frontal.jpg',
 'D:\\Datasets\\CheXpert-v1.0-small\\train\\patient00734\\study2\\view1_frontal.jpg',
 'D:\\Datasets\\CheXpert-v1.0-small\\train\\patient00734\\study1\\view1_frontal.jpg',
 'D:\\Datasets\\CheXpert-v1.0-small\\train\\patient28598\\study3\\view1_frontal.jpg']

In [19]:
train_image_paths.remove('D:\\Datasets\\CheXpert-v1.0-small\\train\\patient00001\\study1\\._view1_frontal.jpg')

In [20]:
import random
random.shuffle(train_image_paths)

In [88]:
train_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
test_ds = tf.data.Dataset.from_tensor_slices(valid_image_paths)

In [22]:
image_count = tf.data.experimental.cardinality(train_ds).numpy()

In [89]:
val_size = int(image_count * 0.99)
train_final_ds = train_ds.skip(val_size)
val_ds = train_ds.take(val_size)

In [24]:
main_df = pd.concat([train_df, valid_df], axis = 0)

In [82]:

def process_path(file_path):
    parts = list(tf.strings.split(file_path, os.path.sep).numpy()[-5:])
    parts = [i.decode() for i in parts]
    min_path = '/'.join(parts)
    label = tf.cast(list(main_df.loc[min_path,label_columns]), dtype = tf.int16)
    
    # Loading the image
    img = tf.io.read_file(file_path)
    
    img = tf.io.decode_jpeg(img, channels = 1)
    # Resizing the image
    img = tf.image.resize(img, get_img_size()[:2])
    img =  tf.image.grayscale_to_rgb(img)
#     img = tf.cast(img, dtype = tf.int16)
    img = tf.keras.applications.mobilenet.preprocess_input(img)
    return img, label

In [90]:
train_final_ds = train_final_ds.map(lambda x: tf.py_function(func = process_path,inp = [x], Tout = (tf.float32, tf.int16)), num_parallel_calls = tf.data.AUTOTUNE)
valid_ds = val_ds.map(lambda x: tf.py_function(func = process_path,inp = [x], Tout = (tf.float32, tf.int16)), num_parallel_calls = tf.data.AUTOTUNE)
test_ds = test_ds.map(lambda x: tf.py_function(func = process_path,inp = [x], Tout = (tf.float32, tf.int16)), num_parallel_calls = tf.data.AUTOTUNE)

In [91]:
batch_size = 4
train = train_final_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
valid = valid_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

test = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [30]:
def get_img_size():
    return (input_height, input_width, channels)

In [49]:
# class SaveBestCheckpoint(tf.keras.callbacks.Callback):
#     def __init__(self):
#         super().__init__()
        

### EfficientNetV2-M

In [29]:
import sys
effnet_path = os.path.join(os.getcwd(), 'automl', 'efficientnetv2')
sys.path.append(effnet_path)
import effnetv2_model

In [31]:
input_height, input_width, channels = (320,320,3)

In [35]:
learning_rate_scheduler = tf.keras.optimizers.schedules.CosineDecayRestarts(initial_learning_rate=0.0001, 
                                                  first_decay_steps=1000,
                                                  alpha = 0.02,
                                                  m_mul=0.9, 
                                                  t_mul = 2  
                                                 )

In [92]:
tf.keras.backend.clear_session()
effnetv2_m = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=get_img_size()),
    effnetv2_model.get_model('efficientnetv2-m', include_top=False, pretrained=False),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(14, activation='sigmoid'),
])
effnetv2_m.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetv2-m (EffNetV2Mo (None, 1280)              53150388  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 14)                17934     
Total params: 53,168,322
Trainable params: 52,876,290
Non-trainable params: 292,032
_________________________________________________________________


In [93]:
optimizer=tf.keras.optimizers.Adam(learning_rate_scheduler,)

effnetv2_m.compile(optimizer=optimizer, loss = tf.keras.losses.CategoricalCrossentropy(), metrics = [tf.keras.metrics.AUC()])

checkpoint_path = './checkpoints/train/efficientNetV2-m'

early_stopping = tf.keras.callbacks.EarlyStopping(min_delta = 0.0001, patience = 3)
#reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor = 0.1, patience = 1)
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_best_only=True)
callbacks = [early_stopping, checkpoint]

checkpoint._supports_tf_logs = False

effnetv2_m.fit(train,validation_data=test, epochs = 2, callbacks=callbacks, verbose = 1)

### EfficientNetV2 - S

In [79]:
import sys
effnet_path = os.path.join(os.getcwd(), 'automl', 'efficientnetv2')
sys.path.append(effnet_path)
import effnetv2_model

In [80]:
learning_rate_scheduler = tf.keras.optimizers.schedules.CosineDecayRestarts(initial_learning_rate=0.0001, 
                                                  first_decay_steps=1000,
                                                  alpha = 0.02,
                                                  m_mul=0.9, 
                                                  t_mul = 2  
                                                 )

In [129]:
input_height, input_width = (320,320)

In [152]:
tf.keras.backend.clear_session()
effnetv2_s = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=[320,320, 3]),
    effnetv2_model.get_model('efficientnetv2-s', include_top=False, pretrained=False),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(14, activation='sigmoid'),
])
effnetv2_s.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetv2-s (EffNetV2Mo (None, 1280)              20331360  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 14)                17934     
Total params: 20,349,294
Trainable params: 20,195,422
Non-trainable params: 153,872
_________________________________________________________________


In [153]:
optimizer=tf.keras.optimizers.Adam(learning_rate_scheduler,)

effnetv2_s.compile(optimizer=optimizer, loss = tf.keras.losses.CategoricalCrossentropy(), metrics = [tf.keras.metrics.AUC()])

In [154]:
checkpoint_path = './checkpoints/train/efficientNetV2-s'

In [155]:
early_stopping = tf.keras.callbacks.EarlyStopping(min_delta = 0.0001, patience = 3)
#reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor = 0.1, patience = 1)
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path)
callbacks = [early_stopping, checkpoint]

In [156]:
effnetv2_s.fit(train,validation_data=test, epochs = 10, callbacks=callbacks, verbose = 1)

Epoch 1/10
13964/13964 [==============================] - 3574s 255ms/step - loss: 6.7576 - auc: 0.7891 - val_loss: 6.8134 - val_auc: 0.7837


INFO:tensorflow:Assets written to: ./checkpoints/train\efficientNetV2-s\assets


INFO:tensorflow:Assets written to: ./checkpoints/train\efficientNetV2-s\assets


Epoch 2/10
13964/13964 [==============================] - 3566s 255ms/step - loss: 6.6119 - auc: 0.8122 - val_loss: 6.7167 - val_auc: 0.8131


INFO:tensorflow:Assets written to: ./checkpoints/train\efficientNetV2-s\assets


...

INFO:tensorflow:Assets written to: ./checkpoints/train\efficientNetV2-s\assets


INFO:tensorflow:Assets written to: ./checkpoints/train\efficientNetV2-s\assets


Epoch 7/10
13964/13964 [==============================] - 3496s 250ms/step - loss: 6.2665 - auc: 0.8557 - val_loss: 6.7373 - val_auc: 0.8380


INFO:tensorflow:Assets written to: ./checkpoints/train\efficientNetV2-s\assets


INFO:tensorflow:Assets written to: ./checkpoints/train\efficientNetV2-s\assets


In [159]:
effnetv2_s.save(r'C:\Users\prans\Python files\Kaggle Competitions\Covid_19_object_detection\saved_models\efficientNetV2-s')

INFO:tensorflow:Assets written to: C:\Users\prans\Python files\Kaggle Competitions\Covid_19_object_detection\saved_models\efficientNetV2-s\assets


INFO:tensorflow:Assets written to: C:\Users\prans\Python files\Kaggle Competitions\Covid_19_object_detection\saved_models\efficientNetV2-s\assets


### EfficientNetB0

In [25]:
learning_rate_scheduler = tf.keras.optimizers.schedules.CosineDecayRestarts(initial_learning_rate=0.0001, 
                                                  first_decay_steps=1000,
                                                  alpha = 0.02,
                                                  m_mul=0.9, 
                                                  t_mul = 2  
                                                 )

In [27]:
input_height, input_width = (320,320)

In [28]:
baseline = tf.keras.applications.EfficientNetB0(include_top=False,
weights = None, input_shape = (input_height, input_width, 1))

In [51]:
efficientNetB0 = tf.keras.models.Sequential([
    baseline,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(14, activation = 'sigmoid')
])

NameError: name 'baseline' is not defined

In [30]:
efficientNetB0.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, 10, 10, 1280)      4048991   
_________________________________________________________________
flatten (Flatten)            (None, 128000)            0         
_________________________________________________________________
dropout (Dropout)            (None, 128000)            0         
_________________________________________________________________
dense (Dense)                (None, 14)                1792014   
Total params: 5,841,005
Trainable params: 5,798,986
Non-trainable params: 42,019
_________________________________________________________________


In [31]:
optimizer=tf.keras.optimizers.Adam(learning_rate_scheduler,)

efficientNetB0.compile(optimizer=optimizer, loss = tf.keras.losses.CategoricalCrossentropy(), metrics = [tf.keras.metrics.AUC()])

In [32]:
checkpoint_path = './checkpoints/train/efficientNetB0'

# ckpt = tf.train.Checkpoint(denseNet121 =  denseNet121, 
#                           optimizer = optimizer)
# ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep = 3)

# # if a checkpoint exists, restore the latest checkpoint
# if ckpt_manager.latest_checkpoint:
#     clpt.restore(ckpt_manager.latest_checkpoint)
#     print('Latest checkpoint restored!!')

In [33]:
early_stopping = tf.keras.callbacks.EarlyStopping(min_delta = 0.0001, patience = 2)
#reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor = 0.1, patience = 1)
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path)
callbacks = [early_stopping, checkpoint]

In [34]:
efficientNetB0.fit(train,validation_data=test, epochs = 3, callbacks=callbacks, verbose = 1)

Epoch 1/3
4190/4190 [==============================] - 1228s 291ms/step - loss: 6.9353 - auc: 0.7374 - val_loss: 6.8294 - val_auc: 0.7315


C:\Users\prans\Python files\Kaggle Competitions\Covid_19_object_detection\env\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: ./checkpoints/train\efficientNetB0\assets
Epoch 2/3
4190/4190 [==============================] - 1184s 282ms/step - loss: 6.7405 - auc: 0.7497 - val_loss: 6.9361 - val_auc: 0.7295


C:\Users\prans\Python files\Kaggle Competitions\Covid_19_object_detection\env\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: ./checkpoints/train\efficientNetB0\assets
Epoch 3/3
4190/4190 [==============================] - 1183s 282ms/step - loss: 6.6921 - auc: 0.7593 - val_loss: 7.0006 - val_auc: 0.7178


C:\Users\prans\Python files\Kaggle Competitions\Covid_19_object_detection\env\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: ./checkpoints/train\efficientNetB0\assets


In [35]:
efficientNetB0.fit(train,validation_data=test, epochs = 5, callbacks=callbacks, verbose = 1)

Epoch 1/5
4190/4190 [==============================] - 1179s 281ms/step - loss: 6.6461 - auc: 0.7661 - val_loss: 7.0076 - val_auc: 0.7187


C:\Users\prans\Python files\Kaggle Competitions\Covid_19_object_detection\env\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: ./checkpoints/train\efficientNetB0\assets
Epoch 2/5
4190/4190 [==============================] - 1190s 284ms/step - loss: 6.6511 - auc: 0.7691 - val_loss: 6.9995 - val_auc: 0.7186


C:\Users\prans\Python files\Kaggle Competitions\Covid_19_object_detection\env\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: ./checkpoints/train\efficientNetB0\assets
Epoch 3/5
4190/4190 [==============================] - 1191s 284ms/step - loss: 6.6076 - auc: 0.7729 - val_loss: 6.9621 - val_auc: 0.7217


...

INFO:tensorflow:Assets written to: ./checkpoints/train\efficientNetB0\assets
Epoch 4/5
4190/4190 [==============================] - 1192s 284ms/step - loss: 6.5601 - auc: 0.7783 - val_loss: 6.9279 - val_auc: 0.7307


C:\Users\prans\Python files\Kaggle Competitions\Covid_19_object_detection\env\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: ./checkpoints/train\efficientNetB0\assets
Epoch 5/5
4190/4190 [==============================] - 1204s 287ms/step - loss: 6.5873 - auc: 0.7776 - val_loss: 6.9296 - val_auc: 0.7286


C:\Users\prans\Python files\Kaggle Competitions\Covid_19_object_detection\env\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: ./checkpoints/train\efficientNetB0\assets


In [40]:
#efficientNetB0.save('.saved_models/efficientNetB0/auc_0.7286/')

C:\Users\prans\Python files\Kaggle Competitions\Covid_19_object_detection\env\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: ./efficientNetB0/auc_0.7286/assets


In [36]:
model = efficientNetB0.load_weights(checkpoint_path)

In [42]:
val_size = int(image_count * 0.4)
train_final_ds = train_ds.skip(val_size)
val_ds = train_ds.take(val_size)

In [43]:
train_final_ds = train_final_ds.map(lambda x: tf.py_function(func = process_path,inp = [x], Tout = (tf.float32, tf.int16)), num_parallel_calls = tf.data.AUTOTUNE)
valid_ds = val_ds.map(lambda x: tf.py_function(func = process_path,inp = [x], Tout = (tf.float32, tf.int16)), num_parallel_calls = tf.data.AUTOTUNE)
#test_ds = test_ds.map(lambda x: tf.py_function(func = process_path,inp = [x], Tout = (tf.float32, tf.int16)), num_parallel_calls = tf.data.AUTOTUNE)

In [44]:
train = train_final_ds.batch(16).prefetch(tf.data.AUTOTUNE)
valid = valid_ds.batch(16).prefetch(tf.data.AUTOTUNE)

In [25]:
test = test_ds.batch(16).prefetch(tf.data.AUTOTUNE)

In [45]:
efficientNetB0.fit(train,validation_data=test, epochs = 3, callbacks=callbacks, verbose = 1)

Epoch 1/3
8379/8379 [==============================] - 2471s 295ms/step - loss: 6.5941 - auc: 0.7792 - val_loss: 6.9704 - val_auc: 0.7317


C:\Users\prans\Python files\Kaggle Competitions\Covid_19_object_detection\env\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: ./checkpoints/train\efficientNetB0\assets
Epoch 2/3
8379/8379 [==============================] - 2489s 297ms/step - loss: 6.5385 - auc: 0.7862 - val_loss: 6.9004 - val_auc: 0.7527


C:\Users\prans\Python files\Kaggle Competitions\Covid_19_object_detection\env\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: ./checkpoints/train\efficientNetB0\assets
Epoch 3/3
8379/8379 [==============================] - 2496s 298ms/step - loss: 6.4531 - auc: 0.7926 - val_loss: 6.9161 - val_auc: 0.7520


C:\Users\prans\Python files\Kaggle Competitions\Covid_19_object_detection\env\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: ./checkpoints/train\efficientNetB0\assets


In [67]:
efficientNetB0.save('./saved_models/efficientNetB0/auc_0.7520/')

C:\Users\prans\Python files\Kaggle Competitions\Covid_19_object_detection\env\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: ./saved_models/efficientNetB0/auc_0.7520/assets


### DenseNet Model

In [25]:
learning_rate_scheduler = tf.keras.optimizers.schedules.CosineDecayRestarts(initial_learning_rate=0.0001, 
                                                  first_decay_steps=1000,
                                                  alpha = 0.02,
                                                  m_mul=0.9, 
                                                  t_mul = 2  
                                                 )

In [52]:
input_height, input_width = (320,320)

In [59]:
baseline = tf.keras.applications.DenseNet121(include_top=False,
weights = None, input_shape = (input_height, input_width, 1))

In [60]:
denseNet121 = tf.keras.models.Sequential([
    baseline,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(14, activation = 'sigmoid')
])

In [61]:
denseNet121.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Functional)     (None, 10, 10, 1024)      7031232   
_________________________________________________________________
flatten_3 (Flatten)          (None, 102400)            0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 102400)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 14)                1433614   
Total params: 8,464,846
Trainable params: 8,381,198
Non-trainable params: 83,648
_________________________________________________________________


In [62]:
optimizer=tf.keras.optimizers.Adam(learning_rate_scheduler)

denseNet121.compile(optimizer=optimizer, loss = tf.keras.losses.CategoricalCrossentropy(), metrics = [tf.keras.metrics.AUC()])

In [63]:
checkpoint_path = './checkpoints/train/denseNet121'

# ckpt = tf.train.Checkpoint(denseNet121 =  denseNet121, 
#                           optimizer = optimizer)
# ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep = 3)

# # if a checkpoint exists, restore the latest checkpoint
# if ckpt_manager.latest_checkpoint:
#     clpt.restore(ckpt_manager.latest_checkpoint)
#     print('Latest checkpoint restored!!')

In [195]:
early_stopping = tf.keras.callbacks.EarlyStopping(min_delta = 0.0001, patience = 2)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor = 0.1, patience = 1)
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path)
callbacks = [early_stopping, reduce_lr, checkpoint]

In [64]:
denseNet121.fit(train,validation_data=valid, epochs = 5, callbacks=callbacks, verbose = 1)

Epoch 1/5
4190/4190 [==============================] - 2635s 626ms/step - loss: 7.0814 - auc_1: 0.7169 - val_loss: 6.8811 - val_auc_1: 0.7283
INFO:tensorflow:Assets written to: ./checkpoints/train\efficientNetB0\assets
Epoch 2/5
4190/4190 [==============================] - 2624s 626ms/step - loss: 6.8443 - auc_1: 0.7333 - val_loss: 6.8266 - val_auc_1: 0.7492
INFO:tensorflow:Assets written to: ./checkpoints/train\efficientNetB0\assets
Epoch 3/5
4190/4190 [==============================] - 2743s 655ms/step - loss: 6.7385 - auc_1: 0.7518 - val_loss: 6.5803 - val_auc_1: 0.7875
INFO:tensorflow:Assets written to: ./checkpoints/train\efficientNetB0\assets
Epoch 4/5
4190/4190 [==============================] - 2740s 654ms/step - loss: 6.6645 - auc_1: 0.7669 - val_loss: 7.5216 - val_auc_1: 0.5725
INFO:tensorflow:Assets written to: ./checkpoints/train\efficientNetB0\assets
Epoch 5/5
4190/4190 [==============================] - 2859s 682ms/step - loss: 6.7132 - auc_1: 0.7677 - val_loss: 6.6320 - 

In [66]:
denseNet121.save('./saved_models/denseNet121/auc_0.7912/')

INFO:tensorflow:Assets written to: ./saved_models/denseNet121/auc_0.7912/assets
